In [323]:
import numpy as np 
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pickle
import random
from copy import copy, deepcopy
from scipy.io import arff
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.spatial.distance import pdist, cdist
from sklearn.metrics.pairwise import euclidean_distances, pairwise_distances
from sklearn.utils import check_random_state
from scipy.spatial import distance
from tslearn.soft_dtw_fast import *
from tslearn.cysax import *
from tslearn.metrics import dtw, dtw_path,dtw_path_from_metric,soft_dtw
from tslearn.utils import *


# Loading Training and Testing Data Source

In [324]:
data1 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension1_TRAIN.arff')
data2 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension2_TRAIN.arff')
data3 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension3_TRAIN.arff')

Test_data1 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension1_TEST.arff')
Test_data2 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension2_TEST.arff')
Test_data3 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension3_TEST.arff')

In [325]:
df_D1 = pd.DataFrame(data1[0])
df_D2 = pd.DataFrame(data2[0])
df_D3 = pd.DataFrame(data3[0])

Test_df_D1 = pd.DataFrame(Test_data1[0])
Test_df_D2 = pd.DataFrame(Test_data2[0])
Test_df_D3 = pd.DataFrame(Test_data3[0])

## Make Over the data, as desired

In [326]:
df_D1 =df_D1.apply(lambda x : x.fillna(method='ffill'), axis=1) 
df_D2 =df_D2.apply(lambda x : x.fillna(method='ffill'), axis=1)
df_D3 =df_D3.apply(lambda x : x.fillna(method='ffill'), axis=1) 

Test_df_D1 =Test_df_D1.apply(lambda x : x.fillna(method='ffill'), axis=1) 
Test_df_D2 =Test_df_D2.apply(lambda x : x.fillna(method='ffill'), axis=1)
Test_df_D3 =Test_df_D3.apply(lambda x : x.fillna(method='ffill'), axis=1) 

In [327]:
df_D1 = df_D1.drop('target', axis=1)
df_D2 = df_D2.drop('target', axis=1)
df_D3 = df_D3.drop('target', axis=1)

Test_df_D1 = Test_df_D1.drop('target', axis=1)
Test_df_D2 = Test_df_D2.drop('target', axis=1)
Test_df_D3 = Test_df_D3.drop('target', axis=1)

In [328]:
Sort_Dim_1 = np.expand_dims(df_D1.values.reshape(-1),axis=1)
Sort_Dim_2 = np.expand_dims(df_D2.values.reshape(-1),axis=1)
Sort_Dim_3 = np.expand_dims(df_D3.values.reshape(-1),axis=1)

Sort_Test_Dim_1 = np.expand_dims(Test_df_D1.values.reshape(-1),axis=1)
Sort_Test_Dim_2 = np.expand_dims(Test_df_D2.values.reshape(-1),axis=1)
Sort_Test_Dim_3 = np.expand_dims(Test_df_D3.values.reshape(-1),axis=1)

In [329]:
Training_View = np.concatenate((Sort_Dim_1, Sort_Dim_2),axis=1)
Training_View = np.concatenate((Training_View, Sort_Dim_3),axis=1)

In [330]:
Training_View

array([[-0.13015 ,  0.071323,  0.899306],
       [-0.183121,  0.083973,  1.157239],
       [-0.234104,  0.085534,  1.320469],
       ...,
       [-0.248272,  0.009383, -1.387357],
       [-0.248272,  0.009383, -1.387357],
       [-0.248272,  0.009383, -1.387357]])

In [331]:
Testing_View = np.concatenate((Sort_Test_Dim_1, Sort_Test_Dim_2),axis=1)
Testing_View = np.concatenate((Testing_View, Sort_Test_Dim_3),axis=1)

In [332]:
finalarray_test_data = Testing_View[:][0:182]

In [333]:
finalarray_test_data.shape

(182, 3)

## Initializing the data

In [369]:
Total_rows = 258804
Sample_Percentage = 0
Number_of_run= 0
Length_RTS = 0
global Sub_Sample_copy_PTS 
Training_Results_List = []
PTS_Results_List = []
Store_Avg_list = []
Counter = 0 
dictionary = dict()
dictionary_RTS = dict()
dictionary_RTS_all = dict()
dictionary_PTS = dict()
dictionary_PTS_for_all_change = dict()
RTS_short_list_Results_List = []
Starting_point = Testing_View[:][0:1]
End_point = Testing_View[:][181:182]
Length_RTS = 118

### Prepared One View

In [335]:
One_view = Training_View[:][:].copy()

In [336]:
One_view.shape # contains whole training data 

(258804, 3)

In [337]:
with open('/Users/arslanaliawan/Desktop/One_view', 'wb') as f:
    
    pickle.dump(One_view, f)

In [338]:
with open('/Users/arslanaliawan/Desktop/One_view', 'rb') as f:
    One_view = pickle.load(f)


In [339]:
One_view.shape

(258804, 3)

### Take Sample of random rows (some specific %) from One_view

In [340]:
delta=0.001

In [341]:
num_rand_instances=int(len(One_view)*delta)

In [342]:
random_instance_selection=np.random.randint(len(One_view), size=num_rand_instances)

In [343]:
len(random_instance_selection)

258

## Get the original instances of Training TS from above indices

In [344]:
a = []
for x in range(len(random_instance_selection)):
    
    Index = random_instance_selection[x]
    a.append(list(One_view[Index]))
Subsample_one_view_array = np.array(a)

In [345]:
Subsample_one_view_array.shape

(258, 3)

In [346]:
with open('/Users/arslanaliawan/Desktop/Subsample_one_view_array', 'wb') as f:
    
    pickle.dump(Subsample_one_view_array, f)

In [347]:
with open('/Users/arslanaliawan/Desktop/Subsample_one_view_array', 'rb') as f:
    Subsample_one_view_array = pickle.load(f)



## Creating Similarity matrix of Sample One_view

In [348]:
def generate_similarity_matrix(Subsample_one_view_array):
    ED_Matrix = distance.cdist(Subsample_one_view_array, Subsample_one_view_array, 'euclidean')
    return ED_Matrix

In [349]:
similarity_output = generate_similarity_matrix(Subsample_one_view_array)

In [350]:
with open('/Users/arslanaliawan/Desktop/similarity_output', 'wb') as f:
    
    pickle.dump(similarity_output, f, protocol=pickle.HIGHEST_PROTOCOL)

In [351]:
with open('/Users/arslanaliawan/Desktop/similarity_output', 'rb') as f:
    similarity_output = pickle.load(f)

In [352]:
similarity_output.shape

(258, 258)

## Code for RTS

In [353]:
def Get_random_row(One_view,Length_RTS):
    
    number_of_rows = One_view.shape[0]
    random_indices = np.random.choice(number_of_rows, size=Length_RTS, replace=False)
    random_rows = One_view[random_indices, :]
    return random_rows

In [354]:
def Adding_start_end_point(random_rows):
    Output_TS = np.concatenate((Starting_point, random_rows),axis=0)
    Output_TS = np.concatenate((Output_TS, End_point),axis=0)
    return Output_TS   

In [367]:
def Calculate_Random_DTW(Output_TS,Length_RTS):
    path, dist = dtw_path(finalarray_test_data, Output_TS)
    if(dist>0 and Length_RTS >100 ):
        
        RTS_short_list_Results_List.append(dist)  
    else:
        RTS_short_list_Results_List.append(dist)

In [368]:
def run_all_methods_RTS(Subsample_one_view_array,Length_RTS):
    
    random_rows = Get_random_row(Subsample_one_view_array,Length_RTS)
    Output_TS = Adding_start_end_point(random_rows)
    Calculate_Random_DTW(Output_TS,Length_RTS) 



## Code for PTS


In [357]:
def Adding_start_end_point_PTS(Com_SM,i):
    
    if(i==1):
        Starting_point = Testing_View[:][0:1]
        End_point = Testing_View[:][181:182]
        SP_Matrix_1 = distance.cdist(Starting_point, Subsample_one_view_array, 'euclidean')
        SP_Matrix_2 = distance.cdist(Starting_point, Starting_point, 'euclidean')
        SP_Matrix_3 = distance.cdist(Starting_point, End_point, 'euclidean')
        EP_Matrix_1 = distance.cdist(End_point, Subsample_one_view_array, 'euclidean')
        EP_Matrix_2 = distance.cdist(End_point, Starting_point, 'euclidean')
        EP_Matrix_3 = distance.cdist(End_point, End_point, 'euclidean')
        SP_Matrix = np.concatenate((SP_Matrix_1, SP_Matrix_2),axis=1)
        SP_Matrix = np.concatenate((SP_Matrix, SP_Matrix_3),axis=1)
        EP_Matrix = np.concatenate((EP_Matrix_1, EP_Matrix_2),axis=1)
        EP_Matrix = np.concatenate((EP_Matrix, EP_Matrix_3),axis=1)
        Com_SM = np.concatenate((similarity_output, SP_Matrix_1),axis=0)
        Com_SM = np.concatenate((Com_SM, EP_Matrix_1),axis=0)
        Com_SM = np.concatenate((Com_SM, SP_Matrix.T),axis=1)
        Com_SM = np.concatenate((Com_SM, EP_Matrix.T),axis=1)
    else:
        p = 182
        Starting_point = Testing_View[:][((p*i)-p):(p*(i-1))+1]
        End_point = Testing_View[:][((p*i)-1):p*i]
        SP_Matrix_1 = distance.cdist(Starting_point, Subsample_one_view_array, 'euclidean')
        SP_Matrix_2 = distance.cdist(Starting_point, Starting_point, 'euclidean')
        SP_Matrix_3 = distance.cdist(Starting_point, End_point, 'euclidean')
        EP_Matrix_1 = distance.cdist(End_point, Subsample_one_view_array, 'euclidean')
        EP_Matrix_2 = distance.cdist(End_point, Starting_point, 'euclidean')
        EP_Matrix_3 = distance.cdist(End_point, End_point, 'euclidean')
        SP_Matrix = np.concatenate((SP_Matrix_1, SP_Matrix_2),axis=1)
        SP_Matrix = np.concatenate((SP_Matrix, SP_Matrix_3),axis=1)
        EP_Matrix = np.concatenate((EP_Matrix_1, EP_Matrix_2),axis=1)
        EP_Matrix = np.concatenate((EP_Matrix, EP_Matrix_3),axis=1)
        Com_SM = np.concatenate((similarity_output, SP_Matrix_1),axis=0)
        Com_SM = np.concatenate((Com_SM, EP_Matrix_1),axis=0)
        Com_SM = np.concatenate((Com_SM, SP_Matrix.T),axis=1)
        Com_SM = np.concatenate((Com_SM, EP_Matrix.T),axis=1)       


    return Com_SM

In [358]:
def generate_graph(SM_PTS):
    G = nx.from_numpy_matrix(SM_PTS, create_using=nx.Graph)
    layout = nx.spring_layout(G)
    sizes = len(SM_PTS)
    return G

In [359]:
def generate_MST(G):
    layout = nx.spring_layout(G)
    T=nx.minimum_spanning_tree(G)
    return T

In [360]:
def generate_Dijkstra(T):
    Dij = nx.dijkstra_path(T,258,259)
    return Dij

In [361]:
def doing_miscellaneous_work(Dijkstra_output):
    global Counter
    a = []
    for x in range(len(Dijkstra_output)):
        Counter = Counter +1 
        Index = Dijkstra_output[x]
        a.append(list(One_view[Index]))
    P_TS = np.array(a)
    return P_TS

In [362]:
def generate_DTW_PTS(Miscellaneous_output):
    path, dist = dtw_path(finalarray_test_data, Miscellaneous_output)
    if(dist>0):
        PTS_Results_List.append(dist)     

In [363]:
def Run_all_fun(SM_PTS):
    
    graph_output = generate_graph(SM_PTS)
    MST_output_1 =  generate_MST(graph_output)
    Dijkstra_output = generate_Dijkstra(MST_output_1)
    Miscellaneous_output = doing_miscellaneous_work(Dijkstra_output)
    generate_DTW_PTS(Miscellaneous_output)


In [370]:
for y in range (1,3):
    
    interation_num = 4
    for i in range (1,interation_num):
        SM_PTS = Adding_start_end_point_PTS(Com_SM,i)
        Run_all_fun(SM_PTS)
        
        run_all_methods_RTS(Subsample_one_view_array,Length_RTS)
        
     
    dictionary_PTS[y] = PTS_Results_List
    PTS_Results_List = []
    dictionary_RTS[y] = RTS_short_list_Results_List
    RTS_short_list_Results_List = []
    

In [371]:
dictionary_PTS

{1: [11.684419403960902, 11.51537774014812, 11.887147622602242],
 2: [11.684419403960902, 11.51537774014812, 11.887147622602242]}

In [372]:
dictionary_RTS

{1: [19.722038308484315, 18.55856130294595, 18.98630637216879],
 2: [18.17262903836858, 19.009718495584895, 19.14864042202174]}